In [83]:
import json
from pymongo import MongoClient
import uuid
from pprint import pprint

In [84]:
# generamos los datos localmente b

clientes = [
    {
        'nombre': 'Pedro',
        'apellidos': 'Cuenca',
        'ID': str(uuid.uuid4().int)[:7],
        'dinero_gastado': 0.0,
        'num_pedidos': 0,
        'pedidos': []
    },
    {
        'nombre': 'Ana',
        'apellidos': 'Martínez',
        'ID': str(uuid.uuid4().int)[:7],
        'dinero_gastado': 0.0,
        'num_pedidos': 0,
        'pedidos': []
    },
    {
        'nombre': 'Alicia',
        'apellidos': 'Romero',
        'ID': str(uuid.uuid4().int)[:7],
        'dinero_gastado': 0.0,
        'num_pedidos': 0,
        'pedidos': []
    },
    # Agrega más clientes si es necesario
]

# Creamos otra lista de  diccionarios con los pedidos

pedidos = [
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-18',
        'codigo_cliente': clientes[0]['ID'],
        'importe_total': 100.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-17',
        'codigo_cliente': clientes[0]['ID'],
        'importe_total': 150.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[1]['ID'],
        'importe_total': 200.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[1]['ID'],
        'importe_total': 50.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[1]['ID'],
        'importe_total': 100.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[2]['ID'],
        'importe_total': 200.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[2]['ID'],
        'importe_total': 300.0
    },
]

In [85]:
# Nos conectamos a la base de datos local que creamos.
client = MongoClient('localhost')
print( client.list_database_names())

['Tienda', 'admin', 'config', 'local']


In [86]:
# Creamos nuestra nueva base de datos a)
db = client['Tienda']
# Creamos las colecciones que necesitamos a)
clientes_col = db['clientes']
pedidos_col = db['pedidos']
# Insertamos los clientes uno a uno c)
for cliente in clientes:
    clientes_col.insert_one(cliente)
# Insertamos todos los pedidos a la vez c)
pedidos_col.insert_many(pedidos)
print('')

In [87]:
# Vamos a ver qué hay en nuestra base de datos.
print( client.list_database_names())
print( db.list_collection_names())
print(clientes_col.count_documents({}))
print(pedidos_col.count_documents({}))

['Tienda', 'admin', 'config', 'local']
['pedidos', 'clientes']
6
14


In [88]:
# Leemos todos los clientes y todos los pedidos d)
clients = clientes_col.find()
peds = pedidos_col.find()

# Recorremos todos los clientes
for cliente in clientes:
    # Leemos los pedidos del cliente d)
    id_cliente = cliente['ID']
    pedidos_cliente = pedidos_col.find({'codigo_cliente': id_cliente})

    # Actualizamos los pedidos del cliente
    cliente_pedidos = []
    dinero_gastado = 0.0
    num_pedidos = 0
    for pedido in pedidos_cliente:
        cliente_pedidos.append(pedido)
        dinero_gastado += pedido['importe_total']
        num_pedidos += 1

    # Actualizamos los datos del cliente en la base de datos: e) 
    clientes_col.update_one(
        {'ID': id_cliente},
        {'$set': {'pedidos': cliente_pedidos,
                  'dinero_gastado':dinero_gastado,
                  'num_pedidos':num_pedidos}}
    )

In [89]:
# Vamos a imprimirlo de forma legible usando pprint
for cliente in clientes_col.find({}): # Se pueden meter los condicionales de mongo.
    pprint(cliente)

{'ID': '7791858',
 '_id': ObjectId('6473c61c216a25c18180a7a1'),
 'apellidos': 'Cuenca',
 'dinero_gastado': 250.0,
 'nombre': 'Pedro',
 'num_pedidos': 2,
 'pedidos': [{'_id': ObjectId('6473c61c216a25c18180a7a4'),
              'codigo_cliente': '7791858',
              'codigo_pedido': '1848185',
              'fecha': '2023-05-18',
              'importe_total': 100.0},
             {'_id': ObjectId('6473c61c216a25c18180a7a5'),
              'codigo_cliente': '7791858',
              'codigo_pedido': '2549635',
              'fecha': '2023-05-17',
              'importe_total': 150.0}]}
{'ID': '1627859',
 '_id': ObjectId('6473c61c216a25c18180a7a2'),
 'apellidos': 'Martínez',
 'dinero_gastado': 350.0,
 'nombre': 'Ana',
 'num_pedidos': 3,
 'pedidos': [{'_id': ObjectId('6473c61c216a25c18180a7a6'),
              'codigo_cliente': '1627859',
              'codigo_pedido': '6811233',
              'fecha': '2023-05-16',
              'importe_total': 200.0},
             {'_id': ObjectId('64

In [90]:
#client.drop_database('Tienda') # Descomentar para eliminar la base de datos.

In [91]:
client.close() # Cerramos la conexión con el cliente